In [1]:
!pip install transformers datasets pypdf2 sentence-transformers
import PyPDF2
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [4]:
%%writefile app.py
import streamlit as st
import PyPDF2
from transformers import pipeline

def extract_text_from_pdf(pdf_path):
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def split_text_into_chunks(text, max_words=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunks.append(" ".join(words[i:i+max_words]))
    return chunks

def summarize_chunks(chunks, summarizer_pipeline):
    summarized_chunks = []
    for chunk in chunks:
        try:
            summarized = summarizer_pipeline(chunk, max_length=512, min_length=30, do_sample=False)
            summarized_chunks.append(summarized[0]['summary_text'])
        except Exception as e:
            summarized_chunks.append(chunk)  # If summarization fails, use original text
    return summarized_chunks

def setup_chatbot():
    qa_pipeline = pipeline("question-answering", model="deepset/xlm-roberta-large-squad2")

    def chatbot(question, chunks):
        best_answer = ""
        best_score = 0
        for chunk in chunks:
            try:
                result = qa_pipeline(question=question, context=chunk)
                if result['score'] > best_score:
                    best_score = result['score']
                    best_answer = result['answer']
            except Exception as e:
                continue
        return best_answer

    return chatbot

def run_gui():
    st.title("Chatbot Layanan Administrasi PerkuliahanBerbasis Multi-Agen AI untuk Pengelolaan Informasi Akademik")
    st.write("Unggah PDF Buku Pedoman Perkuliahan FILKOM dan tanyakan pertanyaan yang ingin ditanyakan")

    uploaded_file = st.file_uploader("Upload PDF", type="pdf")
    if uploaded_file:
        with st.spinner("Extracting text from PDF..."):
            text = extract_text_from_pdf(uploaded_file)

        chunks = split_text_into_chunks(text, max_words=500)

        with st.spinner("Summarizing PDF content..."):
            summarizer = pipeline("summarization", model="google/pegasus-xsum")
            summarized_chunks = summarize_chunks(chunks, summarizer)

        chatbot = setup_chatbot()
        st.success("Chatbot siap digunakan!")

        question = st.text_input("Pertanyaan Anda:")
        if question:
            with st.spinner("Mencari jawaban..."):
                answer = chatbot(question, summarized_chunks)
            st.write("Jawaban")
            st.write(answer)

if __name__ == "__main__":
    run_gui()


Overwriting app.py


# Streamlit Set

In [5]:
from pyngrok import ngrok

ngrok.set_auth_token("2qWofkl8bXBvICrtwtCjxOmSwAx_3VKQJ19APXjoomjrsMCBK")

In [6]:
import os
public_url = ngrok.connect(8501)
os.system("streamlit run app.py &")
public_url

<NgrokTunnel: "https://eb67-35-221-250-151.ngrok-free.app" -> "http://localhost:8501">

# **Evaluasi**

In [ ]:
import time
from transformers import pipeline
import PyPDF2
from google.colab import files
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_text_from_pdf(pdf_path):
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def split_text_into_chunks(text, max_words=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunks.append(" ".join(words[i:i+max_words]))
    return chunks

def summarize_chunks(chunks, summarizer_pipeline):
    summarized_chunks = []
    for chunk in chunks:
        try:
            summarized = summarizer_pipeline(chunk, max_length=512, min_length=30, do_sample=False)
            summarized_chunks.append(summarized[0]['summary_text'])
        except Exception as e:
            summarized_chunks.append(chunk)
    return summarized_chunks

def setup_chatbot():
    qa_pipeline = pipeline("question-answering", model="deepset/xlm-roberta-large-squad2")

    def chatbot(question, chunks):
        best_answer = ""
        best_score = 0
        for chunk in chunks:
            try:
                result = qa_pipeline(question=question, context=chunk)
                if result['score'] > best_score:
                    best_score = result['score']
                    best_answer = result['answer']
            except Exception as e:
                continue
        return best_answer

    return chatbot

def evaluate_chatbot(chatbot, summarized_chunks):
    print("\n=== Evaluasi Chatbot ===")

    question = "siapa dekan filkom ub?"
    expected_answer = "WAYAN FIRDAUS MAHMUDY"

    start_time = time.time()
    answer = chatbot(question, summarized_chunks)
    response_time = time.time() - start_time

    print(f"\nPertanyaan: {question}")
    print(f"Jawaban Chatbot: {answer}")
    print(f"Jawaban yang Diharapkan: {expected_answer}")
    print(f"Waktu Respons: {response_time:.2f} detik")

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([expected_answer, answer])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    score = similarity * 100
    print(f"Kesamaan Jawaban: {score:.2f}%")

    if score > 75:
        print("\nChatbot memberikan jawaban yang memuaskan!")
    else:
        print("\nJawaban Chatbot kurang memuaskan.")

def run_chatbot():
    uploaded = files.upload()

    pdf_path = list(uploaded.keys())[0]
    text = extract_text_from_pdf(pdf_path)

    chunks = split_text_into_chunks(text, max_words=500)

    summarizer = pipeline("summarization", model="google/pegasus-xsum")  # Model publik untuk summarization
    summarized_chunks = summarize_chunks(chunks, summarizer)

    chatbot = setup_chatbot()

    print("Chatbot sudah siap! Silakan ajukan pertanyaan tentang isi PDF.\n")
    while True:
        question = input("Anda: ")
        if question.lower() in ['exit', 'quit']:
            print("Keluar dari chatbot. Sampai jumpa!")
            break
        answer = chatbot(question, summarized_chunks)
        print(f"Chatbot: {answer}")

    evaluate_chatbot(chatbot, summarized_chunks)

run_chatbot()
